# Markov Localization

## Introduction

This notebook covers the material explained in the lesson about Markov Localization, an essential technique for achieving precise localization in autonomous vehicles (AVs). As AVs navigate through complex environments, accurate and reliable localization is as crucial as a driver knowing their exact location on the road.

### The Need for Accurate Localization

Autonomous vehicles rely on accurate localization to navigate safely and efficiently. Incorrect localization can lead to navigation errors, compromising the safety of the vehicle and its surroundings. Thus, ensuring the vehicle knows its exact position and orientation relative to a map is foundational for autonomous driving technologies.

### Challenges in Localization

Effective localization faces several challenges:

- **Dynamic Environments:** The environment around a vehicle is constantly changing. Moving objects like pedestrians and other vehicles, along with varying weather conditions, can affect sensor reliability and accuracy.

- **GPS Limitations:** While GPS provides valuable location data, its reliability can decrease significantly in urban areas with tall buildings (known as urban canyons) or in regions where satellite signals are obstructed.

### Overcoming the challenges

Markov Localization offers a robust solution to these challenges. This method integrates sensor data and pre-existing map information to estimate the most probable vehicle location under uncertain conditions. By continuously updating its state with incoming sensor data, Markov Localization helps maintain accurate positioning even when GPS data is unreliable or unavailable.

## Understaing Localization

### Important Notations


To formalize the localization problem we will rely on the following notations:

- $m$: Represents the digital map, which could be a grid map, a feature map, or a map of landmarks.

- $\mathbf{z}_{t:1}$: Represents A sequence of observations from time $1$ to $t$, possibly including range measurements, bearings, or images. For instance, with a radar sensor in a 1D map, we obtain the distance $x$ to landmarks; in a 2D map, distances $x, y$ are obtained. 
    $$
        \mathbf{z}_{t:1} = [\mathbf{z}_t, ...,\mathbf{z}_i, .... , \mathbf{z}_1] \\
        \mathbf{z}_i =  [z_i^1, ..., z_i^k]
    $$

    Where $z_i$ is the observation vector at time $i$, and $z_i^k$ is the $k^{th}$ observation of $z_i$

- $\mathbf{u}_{t:1}$: Represents the control vector containing control inputs from time $1$ to $t$:

    $$
        \mathbf{u}_{t:1} = [u_t, .... u_1]
    $$

- $\mathbf{x}_t$: Represents the state vector representing the vehicle's pose (position and orientation) at time $t$:

    $$
        \mathbf{x}_t = [x_t,y_t,\theta_t]
    $$



### The problem


Given a map $m$, observation sequence $z_{t:1}$ and a control vector $\mathbf{u}_{t:1}$, the localization problem is to determine the probability or belief $bel(\mathbf{x}_t)$ of the vehicle being in a particular state $\mathbf{x}_t$:

$$
    bel(\mathbf{x}_t) = P(\mathbf{x}_t | \mathbf{z}_{t:1}, \mathbf{u}_{t:1}, m)
$$

This probability is represented as a vector in a 1D map or a matrix in a 2D map, illustrating the vehicle's possible locations across the map dimensions, as follows:

- **1D representation:**
    $$
        bel(\mathbf{x}_t) = [bel(x_t=0), bel(x_t=1), ..., bel(x_t=M)]
    $$

- **2D Case:**
    $$
        bel(\mathbf{x}_t) = \begin{bmatrix}
                                bel(x_t=0, y_t=0) & \cdots & bel(x_t=M, y_t=0) \\
                                    \vdots & \ddots & \vdots \\
                                bel(x_t=0, y_t=N) & \cdots & bel(x_t=M, y_t=N) \\
                                
                            \end{bmatrix}
    $$

## Bayes' Rule


To assess the aformention problem, Markov Localization relies on the Bayes' rule. This is a fundamental theorem in probability theory that allows us to update our beliefs based on new evidence. It is central to numerous applications in statistical inference, and in the context of localization for autonomous vehicles, it plays a pivotal role in updating the belief state of a vehicle regarding its position.

### Mathematical Formulation

Bayes' rule can be mathematically formulated as follows:

$$
    P(A|B) = \frac{P(B|A) \times P(A)}{P(B)}
$$

Where:

- $P(A | B)$ is the probability of event $A$ given that event $B$ has occurred.
- $P(B | A)$ is the probability of observing event $B$ given that event $A$ is true.
- $P(A)$ is the prior probability of event $A$ happening.
- $P(B)$ is the total probability of observing event $B$.

In the localization scenario:

- $A$ represents the hypothesis about the vehicle’s location.
- $B$ represents the observation data from sensors.

This way Bayes' rule can be reinterpreted as follows:

$$
    P(\text{location}|\text{observation}) = \frac{P(\text{observation}|\text{location}) \times P(\text{location})}{P(\text{observation})}
$$

Where:

- $P(\text{location}|\text{observation})$ is the normalized probability of a position given an observation.
- $P(\text{observation}|\text{location})$ is the probability of an observation given a position (likelihood).
- $P(\text{location})$ is the **prior** probability of a position.
- $P(\text{observation})$ is the total probability of an observation.

## Requirements for Localization

Effective localization in autonomous vehicles is essential for accurate navigation, especially in dynamic environments where precision is crucial for safety and operational efficiency. 

Let's consider that our observations come from a lidar, that this lidar has sample rate of 10 Hz and that we have recorded for 8 hours. How much data is stored in $\mathbf{z}_{t:1}$?

In [ ]:
import sys

import numpy as np
import tools.utils as utils
import matplotlib.pyplot as plt

pcd = utils.pcd_from_path("./dataset/example_velodyne.pcd")
pointcloud_size = sys.getsizeof(pcd) # Bytes/frame
sample_rate = 10 # frame/second
hours = 8 # Hours

# Calculate total size
total_size = hours*3600*sample_rate*pointcloud_size

print(f"Total data to process {total_size} Bytes")
print(f"Total data to process {total_size/(1024**3)} Giga Bytes")

Moreover, as more time we have adquiring data, we continue accumlating data on our system which can lead to exploding sizes of data and computational time to process each frame. For localization we need to satisfy the following requirements.

1. Less amount of data possible
2. Amount of data remains constant

To make this possible, we need to find a solution that satisfies the above requirements. The idea is to manipulate the **posterior** $$bel(\mathbf{x}_t)$$ in a way that we can get a recursive state estimator, so we just use the previous beliefs $$bel(\mathbf{x}_{t-1})$$ and current observation $\mathbf{z}_t$ to calculate current beliefs, this means:

$$
    bel(\mathbf{x}_t) = f(bel(\mathbf{x}_{t-1}, \mathbf{z}_t))
$$

## Localization Solution


### Applying Bayes Rule to the Localization Problem


To effectively utilize Bayes' Rule in updating our belief states for localization, we start by refining how we structure our observations. Specifically, we decompose the sequence of observations, into two parts: the most recent observation $z_t$​ and all prior observations $z_{t−1:1}$​. This allows us to reformulate our belief expression as:

$$
    bel(\mathbf{x}_t) = P(\mathbf{x}_t | \mathbf{z}_{t:1}, \mathbf{u}_{t:1}, m) = P(\mathbf{x}_t |  \mathbf{z}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
$$

Given that the current state is $x_t$ and the most recent observation is $z_t$​, we can update our belief about the vehicle's location using Bayes' Rule as follows:
$$
     bel(\mathbf{x}_t) = 
     P(\mathbf{x}_t |  \mathbf{z}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = 
     \frac{P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) \times P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)}
          {P(\mathbf{z}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)}
$$

Where:


- $P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **Prior** or the **Motion Model** or the **Transition Model**
- $P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **likelihood** or the **Observation Model**. (We asume $\mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m$ are given).
- $P(\mathbf{z}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **Normalization term** and is a sum of product of the observation and the motion model ober the states $\mathbf{x}_t$.

To simplify the expression, we define $\eta$ as: 

$$
    \eta = \frac{1}{P(\mathbf{z}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)}
$$

So we obtain:

$$
     bel(\mathbf{x}_t) = \eta \times
                        P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) \times 
                        P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
$$

### Applying Law of Total Probability


To further define $\eta$ we can use the **Law of total probality** that states that:

$$
P(B) = \sum_{i=1}^{\infty}P(B|A_i)P(A_i)
$$

$$
\int P(B|A_i)P(A_i)
$$

This way we can further define $\eta$ as:

$$
\eta = 
\frac{1}{P(\mathbf{z}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)} =
\frac{1}{
            \sum_{i}
                P(\mathbf{z}_t | x_t^{(i)},\mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) \times
                P(x_t^{(i)}|\mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
        }
$$

Where:
 - $P(\mathbf{z}_t | x_t^{(i)},\mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **observation model** over all possible states $x_t^{(i)}$ (positions at time $t$)
 - $P(x_t^{(i)}|\mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **motion model** over all possible states $x_t^{(i)}$ (positions at time $t$)

This means that we just need to define the **motion model** and **observation model** to solve our localization problem. 

## Markov Localization Algorithm

Several algorithms leverage foundational probabilistic rules to solve the localization problem. A widely used approach in robotics is Markov Localization, which effectively handles the uncertainties inherent in real-world environments.

Markov Localization, is a versatile algorithm that uses a form of the Bayes' filter to estimate the position of a robot or vehicle within a known map. It integrates sensory information and movement commands over time to update the belief about the robot's location.

### Markov Assumption

Markov Localization comes from using the bayes filter and applying the Markov Assumption to define the **motion** and **observation** model. This assumption states thet the probability distribution of future states is dependent only upon the current state and not othe preeding states. This can be expressed as: 

$$
    P(x_t | x_{t-1}, x_{t-2}, \cdots, x_0) = P(x_t | x_{t-1})
$$

**Python Example**

A simple example illustrating the Markov assumption in a robotics context involves simulating a robot's movements in a grid-like environment. Let's create a scenario where a robot can move in a deterministic environment with a limited set of actions: move forward, turn left, or turn right. We'll assume the state of the robot is defined only by its current position and orientation, and the future state is dependent solely on its current state and the action taken, which is a key aspect of the Markov assumption.

In [ ]:
# Transition function for the robot
def transition(state, action):
    row, col, orientation = state
    idx = orientations.index(orientation)
    
    if action == 'move forward':
        if orientation == 'up' and row > 0:
            return (row - 1, col, orientation)
        elif orientation == 'right' and col < grid_size[1] - 1:
            return (row, col + 1, orientation)
        elif orientation == 'down' and row < grid_size[0] - 1:
            return (row + 1, col, orientation)
        elif orientation == 'left' and col > 0:
            return (row, col - 1, orientation)
        else:
            return state  # No movement if it leads out of bounds
    elif action == 'turn left':
        new_idx = (idx - 1) % 4
        return (row, col, orientations[new_idx])
    elif action == 'turn right':
        new_idx = (idx + 1) % 4
        return (row, col, orientations[new_idx])
    
    return state

# Simulate the robot's movement
def simulate_robot(steps):
    current_state = start_state
    path = [current_state]
    for _ in range(steps):
        action = np.random.choice(actions)
        current_state = transition(current_state, action)
        path.append(current_state)
    return path


grid_size = (5, 5)

# Possible orientations for the robot: up, right, down, left (clockwise)
orientations = ['up', 'right', 'down', 'left']

# Actions the robot can take
actions = ['move forward', 'turn left', 'turn right']

# Start state (row, column, orientation)
start_state = (2, 2, 'up')  # Middle of the grid, facing up


# Simulate robot's movement for 10 steps
robot_path = simulate_robot(10)
print(f"Step\t\tPosition\tOrientation")
for t, step in enumerate(robot_path):
    print(f"{t}\t\t({step[0]}, {step[1]})\t\t{step[2]}")

### Initialization of the Localization

The initialization phase is crucial as it sets the starting point for the localization process. It begins with establishing a **prior** belief ($P(\text{Location})$) about the vehicle's location, which significantly influences the convergence of most localization algorithms.

#### Initialize Priors in a 1D environment

To illustrate the initialization of priors in a Bayes Filter for localization, consider a simplified scenario:

Suppose we have a 1D map that extends from 0 to 25 meters, with landmarks located at $x=5.0$, $x=10.0$, and $x=20.0$ meters. We assume a position standard deviation ($\sigma_x$) of $1.0$ meter. If we know our vehicle is initially positioned at one of these landmarks, we should set up our initial belief state accordingly.

Given the precision of $±1.0$ meters around each landmark, we can establish the probability of the vehicle being at a location next to a landmark as follows:

- Positions within $[4,6]$, $[9,11]$, or $[19,21]$ meters are considered plausible initial locations for the vehicle.
- The probability assigned to these positions is 1.0, while the probability for all other positions is set to 0.

To normalize these probabilities to ensure they sum up to 1.0 across the map, we calculate the total number of positions that could potentially be occupied, which in this case is 9 (3 positions per landmark). Therefore, each of the plausible positions has a normalized probability of $1.09 \approx 0.111$. Thus, our initial belief state is set as follows:

| Position | Belief |
|----------|--------|
| 0        | 0.0    |
| 1        | 0.0    |
| 2        | 0.0    |
| 3        | 0.0    |
| 4        | 0.111  |
| 5        | 0.111  |
| 6        | 0.111  |
| 7        | 0.0    |
| 8        | 0.0    |
| 9        | 0.111  |
| 10       | 0.111  |
| 11       | 0.111  |
| 12       | 0.0    |
| 13       | 0.0    |
| 14       | 0.0    |
| 15       | 0.0    |
| 16       | 0.0    |
| 17       | 0.0    |
| 18       | 0.0    |
| 19       | 0.111  |
| 20       | 0.111  |
| 21       | 0.111  |
| 22       | 0.0    |
| 23       | 0.0    |
| 24       | 0.0    |
| 25       | 0.0    |

##### Python example

Let's implement the Python function <code>initialize_priors_1d</code>, to set up the initial beliefs about vehicle location on a 1D map. This function considers the positions of landmarks, the standard deviation of location beliefs, the total map size, and the discretization interval of the map, $\Delta x$.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
 
%matplotlib inline

def print_beliefs(beliefs:np.array, map_size: float)->None:
    """
    Function to format print our beliefs

    Args:
        beliefs (np.array): beliefs to be printed
    """
    print("position\tbelief")
    num_points = (len(beliefs)-1)
    delta_x = map_size/num_points
    for position, belief in enumerate(beliefs):
        print(f"{position*delta_x:.2f} m\t\t{belief[0]:.3f}")


def initialize_priors_1d(map_size:float, landmark_positions:np.array, position_stdev:float, delta_x:float=1.0)->np.array:
    """
    Function to initialize the beliefs that the car is in a given position on the map, considering map discretization.

    Args:
        map_size (int): Size of the map in meters.
        landmark_positions (np.array): Positions on the map where landmarks are located, in meters.
        position_stdev (float): Standard deviation of the position, defining the uncertainty around the landmarks.
        delta_x (float): Discretization term for the map, representing the interval between points.

    Returns:
        np.array: Array with the initialized belief of the car being in a given position.
    """
    # Calculate the number of discrete points on the map
    num_points = int(map_size / delta_x) + 1  # +1 to include the zero index

    # Intialize the prior belief state with zeros
    priors = np.zeros((num_points,1))

    # Convert landmark positions to discrete indices
    discrete_landmarks = (landmark_positions / delta_x).astype(int)
    
    # Calculate indices around each landmark within the position standard deviation
    for landmark in discrete_landmarks:
        # Generate index offsets
        indices = np.arange(landmark - position_stdev, landmark + position_stdev + 1)
        # Ensure indices are within map boundaries
        valid_indices = indices[(indices >= 0) & (indices < num_points)]
        # Update priors array
        priors[valid_indices] += 1

    # Normalize the priors array
    priors /= np.sum(priors)

    return priors


# Define map parameters
map_size = 25  # map size in meters
delta_x = 0.5  # discretization interval in meters
landmarks_positions = np.array([5, 10, 20])  # landmark positions in meters
position_stdev = 2  # position standard deviation in terms of discrete steps

# Get our priors belief
priors = initialize_priors_1d(map_size, landmarks_positions, position_stdev, delta_x)

# Plotting the belief state
xs = np.arange(0,len(priors))*delta_x
ys = priors.flatten()

plt.figure(figsize=(12, 4))
plt.bar(xs,ys)
plt.title('Initial Belief of Vehicles Location')
plt.xlabel('Position (meters)')
plt.ylabel('Probability')
plt.grid(True)
plt.show()

#### Python example with 2D Initialization

For the 2D map scenario, let’s extend our method with the <code>initialize_priors_2d function</code>. This function configures the initial belief state for an autonomous vehicle navigating a 2D space. It assigns probabilities based on the proximity to known landmarks, factoring in a specified standard deviation for positional uncertainty around each landmark. Similar to the 1D version, the map is divided into discrete segments according to a given discretization scale, $\Delta_{xy}$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sympy

def initialize_priors_2d(map_size: tuple, landmark_positions: np.array, position_stdev: float, delta_m: float=1.0) -> np.array:
    """
    Initialize the prior beliefs about the vehicle's position on a 2D map, considering the discretization of the map.

    Args:
        map_size (tuple): Size of the map in meters (width, height).
        landmark_positions (np.array): Array of landmark positions on the map [(x1, y1), (x2, y2), ...].
        position_stdev (float): Standard deviation around each landmark, defining the uncertainty.
        delta_ms (float): The distance between discrete positions in the map..

    Returns:
        np.array: 2D array with initialized beliefs of the vehicle being at various positions.
    """
    # Calculate the number of discrete points along each dimension
    num_points_x = int(map_size[0] / delta_m) + 1
    num_points_y = int(map_size[1] / delta_m) + 1

    # Initialize the prior belief state with zeros
    priors = np.zeros((num_points_x, num_points_y))

    # Create meshgrid for indices
    xs, ys = np.meshgrid(np.arange(num_points_x), np.arange(num_points_y), indexing='ij')
    
    # Convert to xs and ys to map positions
    xs = xs*delta_m
    ys = ys*delta_m
    
    # Update the priors for each landmark
    for (landmark_x, landmark_y) in landmark_positions:
        # Calculate distances from each grid point to the landmark
        distance = np.sqrt((xs - landmark_x)**2 + (ys - landmark_y)**2)
        # Set to 1 in each posible location of our vehicle
        priors[distance<=position_stdev] = 1.0

    # Normalize the priors to sum to one
    priors /= np.sum(priors)

    return priors

# Define parameters for the 2D map
map_size = (25, 25)  # size in meters (width, height)
delta_m = 0.2  # The distance between discrete positions in the map.
landmark_positions = np.array([[2, 2], [10, 10], [25, 15]])  # landmark positions in meters
position_stdev = 1.5  # standard deviation

# Initialize the priors
priors = initialize_priors_2d(map_size, landmark_positions, position_stdev, delta_m)

# Plotting the belief state
extent = [0, map_size[0], 0, map_size[1]]
plt.figure(figsize=(10, 10))
plt.imshow(priors.T, origin='lower', extent=extent)
plt.colorbar(label='Probability')
plt.title('Initial 2D Belief of Vehicles Location')
plt.xlabel('Position X (meters)')
plt.ylabel('Position Y (meters)')
plt.grid(True)
plt.show()

### Defining the Motion Model

The Markov Localization algorithm employs Bayes' rule to estimate the position of a robot within a specific environment. This probabilistic approach relies fundamentally on two components: the **motion model** and the **observation model**. The motion model is designed to predict the vehicle's state, $\mathbf{x}_t$, at any given time $t$, using its prior state, the actions undertaken ($\mathbf{u}_{t:1}$), and previously observed data ($\mathbf{z}_{t−1:1}$), within the framework of a known map $m$.

Incorporating the Markov assumption to simplify our model, and applying the law of total probability, we derive the following expression:

$$
P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \int P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) P(\mathbf{x}_{t-1}|  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) d\mathbf{x}_{t-1}
$$

In this equation:

- $P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **state transition** probability, which models the probability of moving to state $x_t$ from state $x_{t−1}$​ given the action $u_t$​ and the map $m$. 
    
- $P(\mathbf{x}_{t-1}|  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)$ is the **recursive Bayesian update**, reflecting the posterior probability of the robot's state at time $t−1$ after considering all previous sensor observations and actions up to $t−1$.


Regarding the localization problem, we can levarage on the properties of the motion model and its dependency on previous states to formulate the following assumptions:

1. Since we (hypothetically) know the state of the system at timestep $t-1$, the past observations $\mathbf{z}_{t-1:1}$ and controls $\mathbf{u}_{t:1}$ up to $t-1$ would not provide additional information to estimate the posterior $x_t$, as they were already used to estimate $\mathbf{x}_{t-1}$. This means, we can simplify the model as: 

    $$
    P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{u}_{t}, m)
    $$

2. Since $u_t$ is "in the future" with reference to $x_{t-1}$, $u_t$ does not provide additional information about $x_{t-1}$. This means that we can state that:

    $$
        P(\mathbf{x}_{t-1}|  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = P(\mathbf{x}_{t-1}|\mathbf{z}_{t-1}  \mathbf{z}_{t-2:1}, \mathbf{u}_{t-1:1}, m)
    $$ 

    Notice that we separated $\mathbf{z}_{t-1:1}$ in $\mathbf{z}_{t}$, $\mathbf{z}_{t-2:1}$

Combining these assumptions, we can express the motion model as:

$$
P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \int P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{u}_{t}, m)P(\mathbf{x}_{t-1}|\mathbf{z}_{t-1},  \mathbf{z}_{t-2:1}, \mathbf{u}_{t-1:1}, m) d\mathbf{x}_{t-1}
$$

Now remembering our priors distribution:

$$
bel(\mathbf{x}_t ) = P(\mathbf{x}_{t}|\mathbf{z}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
$$

Then for $t-1$
$$
bel(\mathbf{x}_{t-1}) = P(\mathbf{x}_{t-1}|\mathbf{z}_{t-1},  \mathbf{z}_{t-2:1}, \mathbf{u}_{t-1:1}, m)
$$

Finally we can write the motion model as:

$$
P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \int P(\mathbf{x}_t | \mathbf{x}_{t-1}, \mathbf{u}_{t}, m) bel(\mathbf{x}_{t-1}) d\mathbf{x}_{t-1}
$$ 

Or for a discrete environment:

$$
P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \sum_i^{N} P(\mathbf{x}_t | \mathbf{x}^{(i)}_{t-1}, \mathbf{u}_{t}, m) bel(\mathbf{x}^{(i)}_{t-1})
$$ 

Typically the **State Transition Model**, is assumed to be map indenpendant. Under this assumption, the model is primarily concerned with the effect of control actions $u_t$​ on the state transition from $x_{t−1}$ to $x_t$​. This simplification can be formally expressed as:

$$
    P(\mathbf{x}_t | \mathbf{x}^{(i)}_{t-1}, \mathbf{u}_{t}, m) = P(\mathbf{x}_t | \mathbf{x}^{(i)}_{t-1}, \mathbf{u}_{t})
$$

In this example, we will assume that the distances moved by the vehicle are normally distributed, with the vehicle's movement ($u_t$​) being the mean and a standard deviation ($\sigma_{u}$​). This forms the basis of our Transition Model:

$$
    P(\mathbf{x}_t | \mathbf{x}^{(i)}_{t-1}, \mathbf{u}_{t}) \sim \mathcal{N}(\mathbf{x}_t-\mathbf{x}^{(i)}_{t-1}; \mathbf{u}_{t}; \sigma_{\mathbf{u}_{t}})
$$

This representation uses a Gaussian distribution to model the probability of transitioning from a previous state $x^{(i)}_{t−1}$​ to a new state $x_t$​, given the movement of the vehicle $u_t$​. It's important to note that this is a simplification; more sophisticated models, such as the kinematic bicycle model, can provide more accurate descriptions of vehicle dynamics, especially in contexts involving turning and complex maneuvers.

#### Python Implementation

Let's implement this model for discrete 1D and 2D environments, demonstrating how the motion model updates our beliefs about vehicle positions based on control inputs and prior probabilities.

##### 1D Example

First, we define our transition model function. This function computes the Gaussian probability density, modeling the likelihood of transitioning from one state to another given a control action. This is a core component of the motion model, representing how likely it is that a vehicle will move to a new position based on the given movement command and inherent movement uncertainty.

In [ ]:
def normpdf(x: np.array, mean: float, std: float) -> np.array:
    """
    Calculate the normal probability density function (Gaussian distribution).

    Args:
        x (np.array): The points at which the Gaussian is evaluated.
        mean (float): The mean (μ) of the Gaussian distribution, representing the expected value.
        std (float): The standard deviation (σ) of the Gaussian distribution, representing the dispersion from the mean.

    Returns:
        np.array: The calculated Gaussian values at each point in x.
    """
    return (1 / np.sqrt(2 * np.pi * std ** 2)) * np.exp(-0.5 * ((x - mean) / std) ** 2)

def transition_model(distance: np.array, movement: float, std: float) -> np.array:
    """
    Use a Gaussian distribution to model the transition probabilities between states based on the movement and its associated uncertainty.

    Args:
        distance (np.array): An array representing the distances from the intended new position for each prior position.
        movement (float): The intended movement which acts as the mean of the Gaussian distribution.
        std (float): The standard deviation of the movement, representing the uncertainty associated with the control input.

    Returns:
        np.array: Transition probabilities calculated using the Gaussian distribution.
    """
    return normpdf(distance, movement, std)

Next, we define the motion model which calculates the updated belief about the vehicle's position after applying the control input. The model sums over all potential prior positions, weighting them by how likely each transition is given the motion model.

In [ ]:
def motion_model(pseudo_position:float, movement:float, priors:np.array, delta_x:float, control_std:float)->float:
    """
    Implementation of the motion model in python. As discussed in the lesson, the probability of the car being in a position given
    a movement and priors. The probability is modeled as a Normal distribution 
    
    N(distance from a given position and a next position, movement, cotrol_std)

    Args:
        pseudo_position (float): Position where we want to calculate the probability 
                                 of landing in this position given a movement, a prior belief and a control standard deviation
        movement (float): Control input
        priors (np.array): Prior beliefs of the position of the car
        control_std (float): Control standard deviation

    Returns:
        float: the probability of landing in a pseudo position given a movement, a prior belief and a control standard deviation
    """
    position_prob = 0
    distances = pseudo_position-np.arange(len(priors))*delta_x
    probs = transition_model(distances, movement, control_std)
    position_prob = probs[None,:]@priors
    return position_prob[0,0]

Let's initialize the map and the priors, and then use the motion model to update our belief state based on a hypothetical control input.

In [ ]:
# Define map parameters
map_size = 25  # map size in meters
delta_x = 0.5  # discretization interval in meters
landmarks_positions = np.array([5, 10, 20])  # landmark positions in meters
position_stdev = 2  # position standard deviation in terms of discrete steps

control_stdev = 1
movement = 2

# Initialize priors based on landmarks
priors = initialize_priors_1d(map_size, landmarks_positions, position_stdev, delta_x)

# Update belief state
position_probs = np.zeros_like(priors)
for i in range(len(priors)):
    x = i * delta_x
    position_probs[i] = motion_model(x, movement, priors, delta_x, control_stdev)

# Plotting the belief state

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.bar(np.arange(len(priors)) * delta_x, priors.flatten(), label="Priors")
ax1.set_title('Prior Belief of Vehicle Location')
ax1.set_xlabel('Position (meters)')
ax1.set_ylabel('Probability')
ax1.grid(True)

ax2.bar(np.arange(len(priors)) * delta_x, position_probs.flatten(), color='green')
ax2.set_title('Posterior Belief of Vehicle Location')
ax2.set_xlabel('Position (meters)')
ax2.set_ylabel('Probability')
ax2.grid(True)

plt.show()

##### 2D example

For a 2D example, we extend the motion model to handle two-dimensional movements. This involves updating our belief about the vehicle's position across a grid of possible locations based on a 2D control input and the previous belief state.

In [ ]:
def motion_model_2D(pseudo_position_x: float, pseudo_position_y: float, movement: float, priors: np.array, delta_m: float, control_std: float) -> float:
    """
    Update the belief about vehicle's position in a 2D map based on prior probability and control input.

    Args:
        pseudo_position_x (float): X-coordinate to evaluate the new probability for.
        pseudo_position_y (float): Y-coordinate to evaluate the new probability for.
        movement (float): The control input specifying the intended movement in both dimensions.
        priors (np.array): The prior probability distribution over positions in the 2D space.
        delta_m (float): The distance between discrete positions in the map.
        control_std (float): Standard deviation of the control input effect.

    Returns:
        float: Updated probability of the vehicle being at the (pseudo_position_x, pseudo_position_y).
    """
    position_prob = 0
    x_points, y_points = priors.shape
    xs, ys = np.meshgrid(np.arange(x_points), np.arange(y_points), indexing='ij')
    
    xs = xs * delta_m
    ys = ys * delta_m
    
    # We consider that we only move forward
    dx = pseudo_position_x - xs
    dy = pseudo_position_y - ys
    distances = np.sqrt((dx)**2 + (dy)**2)

    # We consider that we only move forward
    distances[(dx<0) & (dy<0)] *= -1
    
    probs = transition_model(distances, movement, control_std)
    position_prob = np.sum(probs * priors)
    return position_prob

# Initialize and update the 2D map
map_size = (25, 25)  # size in meters (width, height)
delta_m = 0.5  # discretization interval in meters
landmark_positions = np.array([[5, 5], [10, 10], [20, 20]])  # landmark positions in meters
position_stdev = 1.5  # position standard deviation in terms of discrete steps

control_stdev = 1
movement = 1  # Movement in both x and y directions

# Initialize priors based on landmarks
priors = initialize_priors_2d(map_size, landmark_positions, position_stdev, delta_m)

# Update belief state for 2D
position_probs = np.zeros_like(priors)
x_points, y_points = position_probs.shape

for i in range(x_points):
    for j in range(y_points):
        x = i * delta_m
        y = j * delta_m
        position_probs[i, j] = motion_model_2D(x, y, movement, priors, delta_m, control_stdev)

# Plotting the 2D belief state
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
extent = [0, map_size[0], 0, map_size[1]]

im1 = ax1.imshow(priors, origin="lower", extent=extent, cmap='hot')
ax1.set_title("Prior Belief of Vehicle Location")
ax1.grid(True)
fig.colorbar(im1, ax=ax1, orientation='horizontal', pad=0.08)

im2 = ax2.imshow(position_probs, origin='lower', extent=extent, cmap='hot')
ax2.set_title("Posterior Belief of Vehicle Location")
ax2.grid(True)
fig.colorbar(im2, ax=ax2, orientation='horizontal', pad=0.08)

plt.tight_layout()
plt.show()

### Defining the Observation Model

Let's examine the observation model defined by the probability:

$$
P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
$$

Applying the Markov Assumption simplifies our model considerably. Given that the control inputs $\mathbf{u}_{t:1}$ and previous observations $\mathbf{z}_{t-1:1}$ are utilized to estimate the current state $\mathbf{x}_t$, we can reduce the model to:

$$
P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = P(\mathbf{z}_t |  \mathbf{x}_{t}, m)
$$

Here, the model depends only on the current state $\mathbf{x}_t$ and the map $m$. Note that $\mathbf{z}_t$ represents an observation vector, comprising independently measured observations:

$$
    \mathbf{z}_t = [z_t^1,\cdots, z_t^k]
$$

Therefore, the obeservation molde can be expresed as:

$$
    P(\mathbf{z}_t |  \mathbf{x}_{t}, m) = P(z_t^1,\cdots, z_t^k |  \mathbf{x}_{t}, m) = \prod_{i=1}^k P(z^i_t |  \mathbf{x}_{t}, m)
$$

#### Pseudo ranges

Focus on $P(z^i_t | \mathbf{x}_{t}, m)$, which is the probability of obtaining a measurement $z^i_t$ from state $\mathbf{x}_{t}$. This probability largely depends on the sensor used and its measurements. We implement **sensor models** or **measurement models** to predict a theoretical measurement $\mathbf{z}^*_t$ given the state $\mathbf{x}_{t}$. Let's consider a sensor that measures the distance to known landmarks as depicted below:

<p align="center">
    <img src="https://github.com/dimatura/pypcd/assets/27258035/42f988d5-6523-4324-a779-6a31588fd61f" width="40%" height="auto" >
</p>

Key assumptions for our model include:

- The observation noise follows a Gaussian distribution with a standard deviation $\sigma_{\mathbf{z}_t} = 1m$.
- The measurement range is between $0m$ and $100m$..
- Measurements are taken only in the forward direction.
- The theoretical measurement or pseudo ranges $\mathbf{z}^*_t$ are derived from the state $\mathbf{x}_{t}$ and the map $m$.

##### Example Scenario

Consider the scenario where a car is at $x=5m$, with landmarks at positions $[0, 15, 37, 45, 53, 60, 71]$ m. The theoretical pseudo ranges that the sensor should detect, considering only forward measurements, are:

- No measurement ot landmark at $0m$ as we only sense in forward direction
- $10m$ to landarmk at $15m$
- $32m$ to landarmk at $37m$
- $40m$ to landarmk at $45m$
- $48m$ to landarmk at $53m$
- $55m$ to landarmk at $60m$
- $66m$ to landarmk at $71m$

Therefore, the psuedo ranges are:

$$
    \mathbf{z}^*_t = [10, 32, 40, 48, 55, 66] m
$$

<p align="center">
    <img src="https://github.com/Gaurang-1402/Drona/assets/27258035/f64ee37a-7037-4e7c-a191-38f023d33c7e" width="40%" height="auto" >
</p>


#### Python Implementation

Let's implement our pseudo range estimator. This function calculates pseudo ranges based on the position of the car and known landmark positions. It returns distances only for landmarks that are forward of the vehicle's position; landmarks located behind the vehicle return <code>NaN</code>.

In [ ]:
def pseudo_range_estimator(landmark_positions:np.array, pseudo_position:float)->np.array:
    """
    Function that estimates the pseudo ranges given a position. Remember that a pseudo range
    is the distance the car would observe if its state is pseudo position. 

    Args:
        landmark_positions (np.array): Positions in the map where landmarks are located
        pseudo_position (float): Position where the car could be

    Returns:
        np.array: estimated pseudo ranges
    """
    if len(landmark_positions.shape) == 1:
        landmark_positions = landmark_positions[:, np.newaxis]

    # Calculate ranges as the distances to the landmarks
    diff = landmark_positions - pseudo_position    
    pseudo_ranges = np.sqrt(np.sum(diff**2,axis=1))
    
    # apply a mask to return np.nan to backward measurements.
    # In a 2D case backward measurmentes are mesuraments 
    # whose x and y components are negative.
    mask = np.all(diff<0, axis=1)
    pseudo_ranges[mask] = np.nan
    
    return pseudo_ranges

landmarks_positions_1D = np.array([5, 10, 20])  # landmark positions in meters in one dimension
landmarks_positions_2D = np.array([[5, 5], [10,10], [20,20]]) # landmark positions in meters in two dimensions

pseudo_position_1D = 10  # x position for a 1D example
pseudo_position_2D = np.array([[7, 2]]) # (x,y) position for a 2D example

pseudo_ranges_1D = pseudo_range_estimator(landmarks_positions_1D, pseudo_position_1D)
pseudo_ranges_2D = pseudo_range_estimator(landmarks_positions_2D, pseudo_position_2D)

print(f"Example one dimension:")
print(f"Lanmarks: {landmarks_positions_1D}")
print(f"Position: {pseudo_position_1D}")
print(f"Pseudo ranges: {pseudo_ranges_1D}")

print(f"Example two dimension:")
print(f"Lanmarks: {landmarks_positions_2D}")
print(f"Position: {pseudo_position_2D}")
print(f"Pseudo ranges: {pseudo_ranges_2D}")

##### Visualization of 2D Pseudo Ranges
The 1D pseudo range estimator function calculates distances from a specified vehicle position to an array of landmarks positioned along a single axis. This is useful for simulations where vehicle and landmark positions are constrained to a linear path.

In [ ]:
# Define map parameters
map_size = 25  # map size in meters
delta_x = 0.5  # discretization interval in meters
landmarks_positions = np.array([5, 10, 20])  # landmark positions in meters
position_stdev = 2  # position standard deviation in terms of discrete steps

# Generate positions and compute pseudo ranges for each position
positions = np.linspace(0, map_size, int(map_size / delta_x) + 1)
pseudo_ranges_list = [pseudo_range_estimator(landmarks_positions, x) for x in positions]

# Prepare the updated scatter plot with simplified legend entries
plt.figure(figsize=(12, 6))

# Color mapping for each unique landmark position
unique_landmarks = np.unique(landmarks_positions)
colors = ['blue', 'green', 'orange']
landmark_color_map = {landmark: color for landmark, color in zip(unique_landmarks, colors)}

# Track which labels have been added to the legend
legend_labels_added = set()

# Plot scatter data with consistent color mapping for landmarks
fig = plt.figure()
for idx, x in enumerate(positions):
    pseudo_ranges = pseudo_ranges_list[idx]
    for pseudo_range in pseudo_ranges:
        if np.isnan(pseudo_range):
            continue

        landmark_position = x + pseudo_range
        color = landmark_color_map[landmark_position]
        label = f'Landmark at {landmark_position}m' if color not in legend_labels_added else None
        plt.scatter(x, pseudo_range, color=color, alpha=0.6, label=label)
        if label:
            legend_labels_added.add(color)

plt.title('Pseudo Ranges for Each Position and Landmark')
plt.xlabel('Position (m)')
plt.ylabel('Pseudo Range (m)')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

##### Visualization of 2D Pseudo Ranges
Visualizing 2D pseudo ranges offers a more comprehensive view of the sensor's capabilities and limitations in a planar environment. We use heatmaps to represent the distance from each grid point to nearby landmarks, highlighting variations in sensor reach and obstacles' influence on sensed distances.

In [ ]:

# Initialize and update the 2D map
map_size = (25, 25)  # size in meters (width, height)
delta_m = 0.5  # discretization interval in meters
landmark_positions = np.array([[5, 5], [10,10], [20,20]])  # landmark positions in meters
position_stdev = 1.5  # position standard deviation in terms of discrete steps


# Define a grid
num_x_points = int(map_size[0] / delta_m) + 1
num_y_points = int(map_size[1] / delta_m) + 1

xs = np.linspace(0, map_size[0], num_x_points)
ys = np.linspace(0, map_size[1], num_y_points)

xs, ys = np.meshgrid(xs, ys, indexing='ij')

# Initialize an array to store the minimum pseudo ranges
pseudo_ranges = np.zeros((num_x_points, num_y_points, len(landmark_positions)))

# Calculate the minimum pseudo range for each point on the grid
for i in range(num_x_points):
    for j in range(num_y_points):
        pseudo_position = (xs[i, j], ys[i, j])
        pseudo_ranges[i, j,:] = pseudo_range_estimator(landmark_positions, pseudo_position)

# Plot Psudo ranges
# Initialize the figure for subplots based on the number of landmarks
fig, axes = plt.subplots(nrows=1, ncols=len(landmark_positions), figsize=(18, 5), sharey=True)

# Calculate and plot a heatmap for each landmark
for idx, (landmark, ax) in enumerate(zip(landmark_positions, axes.flatten())):

    # Compute the distances from this landmark to each point on the grid
    distances = pseudo_ranges[:,:,idx]
    
    # Plotting the heatmap
    c = ax.imshow(distances, extent=[0, map_size[0], 0, map_size[1]], origin='lower', cmap='viridis', interpolation='nearest')
    ax.scatter(landmark[0], landmark[1], color='red', s=100, label=f'Landmark {idx+1}')
    ax.set_title(f'Heatmap for Landmark {idx+1}')
    ax.set_xlabel('X coordinate')
    if idx == 0:  # Only add y label to the first subplot to avoid clutter
        ax.set_ylabel('Y coordinate')
    ax.legend()
    fig.colorbar(c, ax=ax, orientation='horizontal', pad=0.12)


Observe that the range estimator does not provide measurements for landmarks once they are positioned behind the vehicle.

#### Implementing the Observation model

To implement the observation model, we utilize the pseudo ranges. Given that the observation noise follows a Gaussian distribution with a standard deviation of $\sigma_{\mathbf{z}_t}$, we can estimate the measurement model distribution as follows:

$$P(z^i_t |  \mathbf{x}_{t}, m) \sim \mathcal{N}(z^i_t,z^{*^i}_t, \sigma_{\mathbf{z}_t})$$

Consequently, the observation model distribution is represented by the product of individual Gaussian distributions:

$$
    P(\mathbf{z}_t |  \mathbf{x}_{t}, m)  = \prod_{i=1}^k P(z^i_t |  \mathbf{x}_{t}, m) \sim \prod_{i=1}^k \mathcal{N}(z^i_t,z^{*^i}_t, \sigma_{\mathbf{z}_t})
$$


At each time step, the observation model performs the following steps for a given pseudo position:

1. Measure the range to landmarks within the map boundaries in the forward direction of the vehicle.
2. Calculate the pseudo range estimate for each landmark by subtracting the pseudo position from the landmark position.
3. Match each pseudo range estimate with its closest observation measurement.
4. For each pair of pseudo range estimate and observation measurement, calculate the probability using the distribution $\mathcal{N}(z^i_t, z^{*^i}t, \sigma{\mathbf{z}_t})$.
5. Compute the product of all probabilities to form the overall observation model distribution: $$\prod_{i=1}^k \mathcal{N}(z^i_t, z^{*^i}_t, \sigma_{\mathbf{z}_t})$$

##### Python Implementation of the observation model

In [ ]:
def observation_model(observations:np.array, pseudo_ranges:np.array, observation_stdev:float)->float:
    """
    Implementation of the observation model. The observation model is the probability of having an observation given
    that we are in a given position that yields the pseudo ranges.

    Args:
        observations (np.array): Observations obtained by our sensor
        pseudo_ranges (np.array): Pseudo ranges obtained in a given pseudo position
        observation_stdev (float): Standard deviation of the noise of our sensor

    Returns:
        (float): Calculated probability
    """
    
    distance_prob = 1
    
    for idx, observation in enumerate(observations):
        pseudo_range_min = 0
        if idx < len(pseudo_ranges):
            pseudo_range_min = pseudo_ranges[idx]
        else:
            pseudo_range_min = np.inf

        distance_prob = distance_prob*normpdf(observation, pseudo_range_min, observation_stdev)

    return distance_prob

##### Visualizing the observation model in one dimension

Let's visualize the probability distribution of the observation model when we meausure. Look how is the distribuition over all states.

In [ ]:
# Define map parameters
map_size = 25  # map size in meters
delta_x = 0.5  # discretization interval in meters
landmarks_positions = np.array([5, 10, 20])  # landmark positions in meters 

# Generate psuedo positions
positions = np.linspace(0, map_size, int(map_size / delta_x) + 1)

# Placeholder to plot the observation model at each pseudo positon
observation_probs = np.zeros_like(positions)

# Step 1. Measure the ranges to landmarks 
sensor_obs = np.asarray([2,8,18])
observation_std = 1.5

for idx, x in enumerate(positions):
    # Step 2. Calculate pseudo ranges
    pseudo_ranges = pseudo_range_estimator(landmarks_positions, x)
    
    # Step 3. Match pseudo ranges, in this case the observations are sorted
    # and as the pseudo ranges doesnt calculate the range to landmarks in 
    # the back, we just need to sort the pseudo ranges and filter the np.nan values
    # then the idx of a psuedo range will match with a measurement
    pseudo_ranges = np.sort(pseudo_ranges[~np.isnan(pseudo_ranges)])

    # Step 4 and 5.
    observation_probs[idx] = observation_model(sensor_obs, pseudo_ranges, observation_std)

# Plots
plt.figure()

# Plot landmarks
plt.vlines(sensor_obs, ymin=0, ymax=np.max(observation_probs), color='black', linestyles='--', label='Landmarks')

# Plotting the belief state
plt.bar(positions, observation_probs.flatten(), color='green', label="Observation model")
plt.title('Probability of getting the observations given a position')
plt.xlabel('Position (meters)')
plt.ylabel('Probability')
plt.grid(True)

plt.legend()
plt.show()

##### Visualizing the observation model in two dimensions.

Now let's visualize the observation model in a two dimension space using heatmaps 

In [ ]:
# Define map parameters
map_size = (25, 25)  # size in meters (width, height)
delta_m = 0.5  # discretization interval in meters
landmark_positions = np.array([[5, 5], [10,10], [20,20]])  # landmark positions in meters
position_stdev = 1.5  # position standard deviation in terms of discrete steps

# Measurements
sensor_obs = np.array([4.84, 18.067])
observation_stdev = 1.0

# Define a grid
num_x_points = int(map_size[0] / delta_m) + 1
num_y_points = int(map_size[1] / delta_m) + 1

# Generate pseudo position in 2D
xs = np.linspace(0, map_size[0], num_x_points)
ys = np.linspace(0, map_size[1], num_y_points)
xs, ys = np.meshgrid(xs, ys, indexing='ij')

# Placeholder to plot the observation model at each pseudo positon
observation_probs = np.zeros((num_x_points, num_y_points))

# Calculate the minimum pseudo range for each point on the grid
for i in range(num_x_points):
    for j in range(num_y_points):
        pseudo_position = (xs[i, j], ys[i, j])
        pseudo_ranges = pseudo_range_estimator(landmark_positions, pseudo_position)
        pseudo_ranges = np.sort(pseudo_ranges[~np.isnan(pseudo_ranges)])
        
        observation_probs[i,j] = observation_model(sensor_obs, pseudo_ranges, observation_stdev)


# Plotting
plt.figure(figsize=(10, 6))
plt.imshow(observation_probs, origin="lower", extent=[0, map_size[0], 0, map_size[1]], cmap='viridis')
plt.scatter(landmark_positions[:, 0], landmark_positions[:, 1], color='red', marker='x', s=100, label='Landmarks')
plt.colorbar()
plt.grid(True)
plt.title("Observation model of Vehicle Location")
plt.xlabel("X Position (m)")
plt.ylabel("Y Position (m)")
plt.legend()
plt.tight_layout()
plt.show()

### Integration of Models

Now equipped with a clear understanding of how to derive and calculate both the motion model and observation model, we can effectively solve the localization problem. The belief in the state at time $t$ can be computed as follows:

$$
bel(\mathbf{x}_t) = 
     \eta \times P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) \times P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m)
$$

With the simplifications previously established:

- **Motion model:**

$$
     P(\mathbf{x}_t | \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \sum_i^{N} P(\mathbf{x}_t | \mathbf{x}^{(i)}_{t-1}, \mathbf{u}_{t}, m) bel(\mathbf{x}^{(i)}_{t-1})
$$

- **Observation Modle:**

$$
     P(\mathbf{z}_t |  \mathbf{x}_{t},  \mathbf{z}_{t-1:1}, \mathbf{u}_{t:1}, m) = \prod_{i=1}^k P(z^i_t |  \mathbf{x}_{t}, m)
$$

Where $\eta$ serves as a normalization constant, calculated as the inverse of the sum of the observation model and motion model probabilities across all states.

Finally the Localization algorithm can formulated as:

1. **Initialization:** Initialize the prior belief about the vehicle's location, assuming proximity to a landmark with known uncertainty.

For each posible state:

2. **Predict the position of the vehicle** Calculate the probability of the position of the vehicle using motion model and previous belief. 

3. **Observe the position of the vehicle** Generate a pseudo-measurement for the current possible state, and use the observation model to estimate the probability distribution of the vehicle's position given the measurements.

4. **Correct the position of the vehicle** Multiply the motion probability and observation probability for that state to derive the unnormalized probability of the vehicle's location.

5. **Repeat** Repeat **2** until you have calculated all unnormalized probablitities for all possible states.

6. **Normalization** Normalize the probabilities for each state by dividing by the sum of all state probabilities.

7. **Update prior belief** Assing the prior belief to your current belief and go to 2. for the next timestep. 

#### Python implementation

Let's implement the complete localization algorithm, processing several measurements across different timesteps. At each timestep and for each measurement, we compute both the motion model and observation model for all possible states, continuously updating our belief regarding the vehicle's location.

##### One dimensional example

In this one-dimensional example, we'll focus on a simplified scenario where both vehicle and landmarks are positioned along a single axis. This simplification allows us to clearly illustrate the algorithm's mechanics without the complexity of multi-dimensional space, making the conceptual details more accessible.

In [ ]:
import tools.utils as utils

# Define map parameters
map_size = 25  # map size in meters
delta_x = 0.5  # discretization interval in meters
landmarks_positions = np.array([3, 9, 14, 23])  # landmark positions in meters
position_stdev = 2  # position standard deviation in terms of discrete steps

movement = 1
control_stdev = 1

sensor_obs = [np.asarray([1,7,12,21]), 
              np.asarray([0,6,11,20]), 
              np.asarray([5,10,19]),
              np.asarray([4,9,18]), 
              np.asarray([3,8,17]), 
              np.asarray([2,7,16]), 
              np.asarray([1,6,15]),
              np.asarray([0,5,14]), 
              np.asarray([4,13]), 
              np.asarray([3,12]), 
              np.asarray([2,11]), 
              np.asarray([1,10]),
              np.asarray([0,9]), 
              np.asarray([8]), 
              np.asarray([7]), 
              np.asarray([6]), 
              np.asarray([5]), 
              np.asarray([4]), 
              np.asarray([3]), 
              np.asarray([2]),
              np.asarray([1]), 
              np.asarray([0]), 
              np.asarray([]), 
              np.asarray([]), 
              np.asarray([])]

observation_stdev = 1
distance_max = 25

# Initialize map positions
xs = np.linspace(0, map_size, int(map_size / delta_x) + 1)

# Initialize priors based on landmarks
priors = initialize_priors_1d(map_size, landmarks_positions, position_stdev, delta_x)

# Posteriors

posteriors_list = [priors.flatten()]

for t, observations in enumerate(sensor_obs):
    
    posteriors = np.zeros_like(priors)

    # step through each pseudo position x (i)
    for idx, pseudo_position in enumerate(xs):

        # Calculate the motion model probability for each x position
        motion_prob = motion_model(pseudo_position, movement, priors, delta_x, control_stdev)
        # Calculate the pseudo ranges
        pseudo_ranges = pseudo_range_estimator(landmarks_positions, pseudo_position)
        pseudo_ranges = np.sort(pseudo_ranges[~np.isnan(pseudo_ranges)])
        # Get the observation probability
        observation_prob = observation_model(observations, pseudo_ranges, observation_stdev)
        posteriors[idx, 0] = motion_prob*observation_prob
    posteriors = posteriors/np.sum(posteriors)

    posteriors_list.append(posteriors)
    priors = posteriors.copy()

utils.animate_markov_loc_results_1D(posteriors_list,xs,[0, 25], [0, 1])



#### Two dimensional example 

In this two-dimensional example, we explore a scenario where the vehicle and landmarks are distributed over a planar area. This setup reflects more realistic conditions as vehicles generally navigate through environments that require understanding both horizontal and vertical spatial relationships. In the 2D model, both the vehicle's movement and the landmarks' positions are considered along two axes (X and Y).

In [ ]:
# Initialize and update the 2D map
map_size = (25, 25)  # size in meters (width, height)
delta_m = 0.5  # discretization interval in meters
landmarks_positions = np.array([[5, 5], [10, 10], [20, 20]])  # landmark positions in meters
position_stdev = 1.5  # position standard deviation in terms of discrete steps

control_stdev = 1
movement = 1  # Movement in both x and y directions


sensor_obs = [[ 1.733, 8.742, 22.811],
              [ 0.811, 6.913, 23.203],
              [ 2.104, 7.419, 20.595],
              [ 2.341, 5.648, 19.796],
              [ 3.880, 17.859],
              [ 2.892, 17.450],
              [ 0.849, 16.421],
              [ 1.321, 15.108],
              [ 2.604, 16.923],
              [ 1.394, 15.197],
              [ 4.041, 12.801],
              [12.52034557],
              [11.97141456],
              [11.19377492],
              [10.66756558],
              [7.2043495  ],
              [7.29971996 ]]

observation_stdev = 1

# Initialize priors based on landmarks
priors = initialize_priors_2d(map_size, landmark_positions, position_stdev, delta_m)

# Update belief state for 2D
posteriors_list = [priors]

x_points, y_points = priors.shape

for t, observations in enumerate(sensor_obs):
    posteriors = np.zeros_like(priors)
    for i in range(x_points):
        x = i * delta_m
        for j in range(y_points):
            y = j * delta_m

            pseudo_position = (x,y)
            # Calculate the motion model probability for each x position
            motion_prob= motion_model_2D(x, y, movement, priors, delta_m, control_stdev)
            # Calculate the pseudo ranges
            pseudo_ranges = pseudo_range_estimator(landmarks_positions, pseudo_position)
            pseudo_ranges = np.sort(pseudo_ranges[~np.isnan(pseudo_ranges)])
            # Get the observation probability
            observation_prob = observation_model(observations, pseudo_ranges, observation_stdev)
            posteriors[i, j] = motion_prob*observation_prob

    posteriors /= np.sum(posteriors)
    priors = posteriors.copy()
    posteriors_list.append(posteriors)

utils.animate_markov_loc_results_2D(posteriors_list, [0, map_size[0]], [0, map_size[1]])